## CSCE 676 :: Data Mining and Analysis :: Texas A&M University :: Spring 2022


# Homework 4: The Final One!

- **100 points [7% of your final grade]**
- **Due April 28, 2022 11:59pm** 
- (*no submissions accepted after May 3, 2022 by 11:59pm*)

*Goals of this homework:* This is an **open-ended** assignment in which you will get to explore embeddings in the context of a large social media dataset. 

*Submission instructions:* You should post your notebook to canvas (look for the homework 4 assignment there). Name your submission **your-uin_hw4.ipynb**, so for example, my submission would be something like **555001234_hw4.ipynb**. Your notebook should be fully executed when you submit ... so run all the cells for us so we can see the output, then submit that. When you are done, download your notebook from colab and submit it to canvas.

*Collaboration declaration:* If you talked to anyone about this homework, please be sure to mention that. Remember to include citations to any sources you use in the homework.

*Write your collaboration/references here*

## Instructions: 

For this homework, you will conduct an independent data mining analysis using embeddings of the congressional Twitter dataset we used in Homework 2. The requirements are threefold:

*   You must use some (or all) of the US Congress Twitter data we provided in Homework 2. You may augment the data if you like, but that is not required. Also feel free to sample just a portion of the data (you do not have to use all of it).
*   You must learn embeddings over the dataset. You may use word2vec to find word embeddings, node2vec to find node embeddings, or some other dense representation. It is up to you, but you must learn some kind of embedding. You may use an existing package to learn these embeddings. Feel free to google around to find the right resource. Some examples include: 

  * word2vec (python, in Gensim) https://radimrehurek.com/gensim/models/word2vec.html
  * doc2vec (python, in Gensim) https://radimrehurek.com/gensim/models/doc2vec.html
  * Node2Vec (python) https://github.com/aditya-grover/node2vec
  * LINE (C++) https://github.com/tangjianpku/LINE
  * DeepWalk (python) https://github.com/phanein/deepwalk, https://pypi.org/project/deepwalk/

* Using these embeddings, you must conduct an interesting analysis of the dataset. For example, are there communities of users you can discover in the node embedding space? Are there outliers in the word embedding space? This is the most interesting part, and you should do your best to make this compelling.


### In the following, you should answer the questions we pose. Then below, you should include your code, including helpful comments and discussion so we can follow your logic.




##  Question 1

What is your compelling data mining application? That is, what is it you aim to discover?

*your answer here*

Use graph node embeddings to predict whether two nodes are exactly connected or not.


##  Question 2

What data did you use? Describe briefly what aspects of the US Congressional data you used.

*your answer here*

I use the congress_members.csv to find all congress members' twitter id as graph nodes, and user_mentions.csv to find the mention relationship between congress members for graph edges. I use 80% of edges for training and 20% of edges for testing.

##  Question 3

What embedding method did you use? And why?

*your answer here*


I use Node2Vec embedding method. It is relatively simple and fast to run on my dataset, and has good graph structure learning abilities.


##  Question 4

How did you apply your embeddings to tackle the problem you posed?

*your answer here*

I first run Node2Vec on the graph with only edges in the training data. After obtaining node embeddings, for each edge in the testing data, I use the cosine similarity of two nodes connected by that edge as the probability of whether these two nodes are connected. If the similarity is larger than 0.5, the prediction is yes, i.e., predicting that those two nodes are connected.


##  Question 5

What did you discover? Provide your results and analysis here.

*your answer here*

I found that the prediction accuracy is less than 0.1, which is very low. I think maybe this is because the number of training samples is small, making it hard for node embeddings to infer all exact neighbors of a node during training.



# Your Analysis

In [1]:
# you should add as many cells as you need below 
!mkdir us-congress-tweets
!wget https://us-congress.s3.amazonaws.com/congress_members.csv -O us-congress-tweets/congress_members.csv
!wget https://us-congress.s3.amazonaws.com/user_mentions.csv -O us-congress-tweets/user_mentions.csv

--2022-04-21 01:10:21--  https://us-congress.s3.amazonaws.com/congress_members.csv
Resolving us-congress.s3.amazonaws.com (us-congress.s3.amazonaws.com)... 54.231.204.185
Connecting to us-congress.s3.amazonaws.com (us-congress.s3.amazonaws.com)|54.231.204.185|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13835 (14K) [text/csv]
Saving to: ‘us-congress-tweets/congress_members.csv’

us-congress-tweets/ 100%[===================>]  13.51K  --.-KB/s    in 0s      

2022-04-21 01:10:22 (79.0 MB/s) - ‘us-congress-tweets/congress_members.csv’ saved [13835/13835]

--2022-04-21 01:10:22--  https://us-congress.s3.amazonaws.com/user_mentions.csv
Resolving us-congress.s3.amazonaws.com (us-congress.s3.amazonaws.com)... 52.217.194.169
Connecting to us-congress.s3.amazonaws.com (us-congress.s3.amazonaws.com)|52.217.194.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56151953 (54M) [text/csv]
Saving to: ‘us-congress-tweets/user_mentions.csv’

us-

In [2]:
import csv
congress_ids = set()
with open('us-congress-tweets/congress_members.csv', mode='r') as csv_file:
  csv_reader = csv.DictReader(csv_file)
  for row in csv_reader:
    id = row['userid']
    congress_ids.add(id)

all_edges = []
with open('us-congress-tweets/user_mentions.csv', mode='r') as csv_file:
  csv_reader = csv.DictReader(csv_file)
  for row in csv_reader:
    src, dst = row['src'], row['dst']
    if src in congress_ids and dst in congress_ids:
      all_edges.append((src, dst, {'weight': 1.0}))

all_nodes = set()
for edge in all_edges:
  src, dst, _ = edge
  all_nodes.add(src)
  all_nodes.add(dst)

print("The total number of nodes is {}, the total number of edges is {}".format(len(all_nodes), len(all_edges)))

The total number of nodes is 254, the total number of edges is 201


In [3]:
import networkx as nx
import random

nx_G = nx.Graph()
random.shuffle(all_edges)
split_bound = int(0.8 * len(all_edges))
train_edges, test_edges = all_edges[:split_bound], all_edges[split_bound:]
nx_G.add_nodes_from(all_nodes)
nx_G.add_edges_from(train_edges)

In [4]:
import numpy as np

class Graph():
	def __init__(self, nx_G, is_directed, p, q):
		self.G = nx_G
		self.is_directed = is_directed
		self.p = p
		self.q = q

	def node2vec_walk(self, walk_length, start_node):
		'''
		Simulate a random walk starting from start node.
		'''
		G = self.G
		alias_nodes = self.alias_nodes
		alias_edges = self.alias_edges

		walk = [start_node]

		while len(walk) < walk_length:
			cur = walk[-1]
			cur_nbrs = sorted(G.neighbors(cur))
			if len(cur_nbrs) > 0:
				if len(walk) == 1:
					walk.append(cur_nbrs[alias_draw(alias_nodes[cur][0], alias_nodes[cur][1])])
				else:
					prev = walk[-2]
					next = cur_nbrs[alias_draw(alias_edges[(prev, cur)][0], 
						alias_edges[(prev, cur)][1])]
					walk.append(next)
			else:
				break

		return walk

	def simulate_walks(self, num_walks, walk_length):
		'''
		Repeatedly simulate random walks from each node.
		'''
		G = self.G
		walks = []
		nodes = list(G.nodes())
		print('Walk iteration:')
		for walk_iter in range(num_walks):
			print('{}/{}'.format(walk_iter+1, num_walks))
			random.shuffle(nodes)
			for node in nodes:
				walks.append(self.node2vec_walk(walk_length=walk_length, start_node=node))

		return walks

	def get_alias_edge(self, src, dst):
		'''
		Get the alias edge setup lists for a given edge.
		'''
		G = self.G
		p = self.p
		q = self.q

		unnormalized_probs = []
		for dst_nbr in sorted(G.neighbors(dst)):
			if dst_nbr == src:
				unnormalized_probs.append(G[dst][dst_nbr]['weight']/p)
			elif G.has_edge(dst_nbr, src):
				unnormalized_probs.append(G[dst][dst_nbr]['weight'])
			else:
				unnormalized_probs.append(G[dst][dst_nbr]['weight']/q)
		norm_const = sum(unnormalized_probs)
		normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]

		return alias_setup(normalized_probs)

	def preprocess_transition_probs(self):
		'''
		Preprocessing of transition probabilities for guiding the random walks.
		'''
		G = self.G
		is_directed = self.is_directed

		alias_nodes = {}
		for node in G.nodes():
			unnormalized_probs = [G[node][nbr]['weight'] for nbr in sorted(G.neighbors(node))]
			norm_const = sum(unnormalized_probs)
			normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]
			alias_nodes[node] = alias_setup(normalized_probs)

		alias_edges = {}
		triads = {}

		if is_directed:
			for edge in G.edges():
				alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
		else:
			for edge in G.edges():
				alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
				alias_edges[(edge[1], edge[0])] = self.get_alias_edge(edge[1], edge[0])

		self.alias_nodes = alias_nodes
		self.alias_edges = alias_edges

		return


def alias_setup(probs):
	'''
	Compute utility lists for non-uniform sampling from discrete distributions.
	Refer to https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
	for details
	'''
	K = len(probs)
	q = np.zeros(K)
	J = np.zeros(K, dtype=np.int)

	smaller = []
	larger = []
	for kk, prob in enumerate(probs):
	    q[kk] = K*prob
	    if q[kk] < 1.0:
	        smaller.append(kk)
	    else:
	        larger.append(kk)

	while len(smaller) > 0 and len(larger) > 0:
	    small = smaller.pop()
	    large = larger.pop()

	    J[small] = large
	    q[large] = q[large] + q[small] - 1.0
	    if q[large] < 1.0:
	        smaller.append(large)
	    else:
	        larger.append(large)

	return J, q

def alias_draw(J, q):
	'''
	Draw sample from a non-uniform discrete distribution using alias sampling.
	'''
	K = len(J)

	kk = int(np.floor(np.random.rand()*K))
	if np.random.rand() < q[kk]:
	    return kk
	else:
	    return J[kk]

In [13]:
G = Graph(nx_G, False, 1.0, 1.0)
G.preprocess_transition_probs()
walks = G.simulate_walks(100, 80)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:112: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Walk iteration:
1/100
2/100
3/100
4/100
5/100
6/100
7/100
8/100
9/100
10/100
11/100
12/100
13/100
14/100
15/100
16/100
17/100
18/100
19/100
20/100
21/100
22/100
23/100
24/100
25/100
26/100
27/100
28/100
29/100
30/100
31/100
32/100
33/100
34/100
35/100
36/100
37/100
38/100
39/100
40/100
41/100
42/100
43/100
44/100
45/100
46/100
47/100
48/100
49/100
50/100
51/100
52/100
53/100
54/100
55/100
56/100
57/100
58/100
59/100
60/100
61/100
62/100
63/100
64/100
65/100
66/100
67/100
68/100
69/100
70/100
71/100
72/100
73/100
74/100
75/100
76/100
77/100
78/100
79/100
80/100
81/100
82/100
83/100
84/100
85/100
86/100
87/100
88/100
89/100
90/100
91/100
92/100
93/100
94/100
95/100
96/100
97/100
98/100
99/100
100/100


In [18]:
from gensim.models import Word2Vec
model = Word2Vec(walks, size=128, window=10, min_count=0, sg=1, workers=8, iter=10)
wv = model.wv

In [19]:
cos_sims = []
for test_edge in test_edges:
  src, dst, _ = test_edge
  src_vec, dst_vec = wv[src], wv[dst]
  cos_sim = src_vec.dot(dst_vec) / np.linalg.norm(src_vec) / np.linalg.norm(dst_vec)
  cos_sims.append(cos_sim)
cos_sims = np.array(cos_sims)
print("The prediction accuracy is {}".format((cos_sims >= 0.5).sum() / len(cos_sims)))

The prediction accuracy is 0.0975609756097561
